<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li>
        <li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item">
        <li><span><a href="#Обучающая-и-тестовая-выборки" data-toc-modified-id="Обучающая-и-тестовая-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучающая и тестовая выборки</a></span></li>
            <li><span><a href="#Вычисление-TD-IDF" data-toc-modified-id="Вычисление-TD-IDF-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Вычисление TD-IDF</a></span></li>
            <li><span><a href="#Обучение-логистической-регрессии-на-TD-IDF" data-toc-modified-id="Обучение-логистической-регрессии-на-TD-IDF-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обучение логистической регрессии на TD-IDF</a></span></li>
            <li><span><a href="#Обучение-LGBMClassifier-на-TD-IDF" data-toc-modified-id="Обучение-LGBMClassifier-на-TD-IDF-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Обучение LGBMClassifier на TD-IDF</a></span></li>
            <li><span><a href="#Обучение-CatBoostClassifier-на-TD-IDF" data-toc-modified-id="Обучение-CatBoostClassifier-на-TD-IDF-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Обучение CatBoostClassifier на TD-IDF</a></span></li></ul></li>
        <li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li>
        <li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul>
</div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import display, HTML
from IPython.display import clear_output

import html
import os.path
import requests
from urllib.parse import urlencode
from joblib import dump, load

# !pip install scikit-learn==0.24.2
# !pip install pandas==1.3.2
# !pip install numpy==1.20.3
# !pip install lightgbm==3.2.1
# !pip install catboost==0.26.1

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_rows = None
pd.options.display.max_columns = None

from pymystem3 import Mystem

import string
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from multiprocessing import Pool

import re

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# Адреса обученных моделей на Яндекс.Диске
MODEL_LR_PUBLIC_KEY = 'https://disk.yandex.ru/d/77gZnQrqOsmZqA'
MODEL_LGBM_PUBLIC_KEY = 'https://disk.yandex.ru/d/tQgentlE-9US-g'
MODEL_CB_PUBLIC_KEY = 'https://disk.yandex.ru/d/Ti8OKj54YNxFZg'

def download_file_from_ydisk(file_url, file_name):
    '''
    Скачивает Яндекс.Диска открытый файл с адресом
    file_url, сохраняет на диск с именем file_name
    '''
    # Полная URL для скачивания собирается из базовой и ULR файла на диске
    base_download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    response_url = requests.get(base_download_url + urlencode(dict(public_key=file_url)), timeout=30)
    # Статус не 200
    if response_url.status_code != 200:
        return 'Невозможно скачать файл: ' + file_url
    # Полная URL для скачивания файла
    full_download_url = response_url.json()['href']
    content_response = requests.get(full_download_url)
    with open(file_name, 'wb') as f:
        f.write(content_response.content)
    return 'Файл ' + file_url + ' скачан и сохранен под именем ' + file_name

def df_info(data_frame, nulls_percentage_round=1, deep_memory_usage=False):
    '''
    Выводит на экран общую информацию о таблице
    Входные параметры:
        data_frame             — таблица типа pd.DataFrame;
        nulls_percentage_round — знак, до которого происходит округление в столбце nulls_percentage отображаемой таблицы;
        deep_memory_usage      — глубокий анализ используемой памяти
    '''
    assert isinstance(data_frame, pd.DataFrame), 'Аргумент data_frame должен иметь тип pd.DataFrame'
    t = pd.concat(
        [
         
            data_frame.isna().sum().to_frame('nulls_count'),
            (100 * data_frame.isna().sum() / len(data_frame)).round(nulls_percentage_round).to_frame('nulls_percentage'),
            data_frame.dtypes.to_frame('dtype')
        ],
        axis=1
    )
    s = 'dtypes:'
    c = data_frame.dtypes.value_counts()
    for i in c.index:
        s += ' {0}({1})'.format(i, c[i])
    m = data_frame.memory_usage(deep=deep_memory_usage).sum()
    if int(m / 1024 ** 3):
        d, b = 1024 ** 3, 'GB'
    elif int(m / 1024 ** 2):
        d, b = 1024 ** 2, 'MB'
    else:
        d, b = 1024, 'KB'
    print(t)
    print('columns: ', data_frame.shape[1], ', rows: ', data_frame.shape[0], sep='')
    print(s)
    print('memory_usage: ', round(m / d, 1), '+ ' if not deep_memory_usage else ' ', b, sep='')

# Набор стоп-слов для английского языка из библиотеки NTLK
NLTK_STOPWORDS = set(nltk_stopwords.words('english') + list(string.punctuation))   

def lemmatize(text):
    '''
    Токенизация и лемматизация с использованием nltk
    '''
    wnl = WordNetLemmatizer()
    return ' '.join([wnl.lemmatize(word) for word in nltk.word_tokenize(text) if word not in NLTK_STOPWORDS])

Загрузим данные из файла.

In [ ]:
# Загрузка данных

# Возможные пути расположения файлов
filepaths = {
    'comments': [
        '/datasets/toxic_comments.csv',
        './toxic_comments.csv',
        'https://code.s3.yandex.net/datasets/toxic_comments.csv'
    ]
}

# Загрузка данных по первому существующему пути
data_list = []
for dfname in filepaths.keys():
    for filepath in filepaths[dfname]:
        try:
            exec(dfname + ' = pd.read_csv(filepath)')
            exec(dfname + f'.name = "{dfname}"')
            exec(f'data_list.append({dfname})')
            print('Таблица ' + dfname + ' из файла ' + filepath + ' загружена в переменную ' + dfname)
            break
        except Exception:
            continue
    else:
        raise SystemExit('Файл с таблицей ' + dfname + ' не найден ни по одному из путей в списке filepaths')    

Таблица comments из файла /datasets/toxic_comments.csv загружена в переменную comments


Начало табилцы.

In [ ]:
comments.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Интернет, родненький. )

In [ ]:
df_info(comments)

       nulls_count  nulls_percentage   dtype
text             0               0.0  object
toxic            0               0.0   int64
columns: 2, rows: 159571
dtypes: object(1) int64(1)
memory_usage: 2.4+ MB


159571 сообщение. Пропусков нет. В исходных текстах используются как заглавные, так и строчные буквы. Это может привести к тому, что одинаковые слова, отличающиеся только регистром букв, могут быть восприняты моделью машинного обучения как разные. Преобразуем регистр всех символов исследуемого корпуса текста к строчному.

In [ ]:
comments.text = comments.text.apply(str.lower)
comments.head(10)

,text,toxic
0,explanation\nwhy the edits made under my usern...,0
1,d'aww! he matches this background colour i'm s...,0
2,"hey man, i'm really not trying to edit war. it...",0
3,"""\nmore\ni can't make any real suggestions on ...",0
4,"you, sir, are my hero. any chance you remember...",0
5,"""\n\ncongratulations from me as well, use the ...",0
6,cocksucker before you piss around on my work,1
7,your vandalism to the matt shirvington article...,0
8,sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Интересно проверить сообщения на наличие дубликатов. Ctrl+C, Ctrl+V никто не отменял, при этом, если одинаковые сообщения в обучающей выборке будут иметь противоположную классификацию, это не пойдет обучению модели на пользу. Выведем первые 10 дублирующихся строк.

In [ ]:
comments[comments.text.duplicated(keep=False)].sort_values(by='text').head(10)

,text,toxic
36974,"""\n\n """"wikipedia isn't perfect, but its model...",0
143080,"""\n\n """"wikipedia isn't perfect, but its model...",0
136461,"""\n\n please do not add nonsense to wikipedia ...",0
78415,"""\n\n please do not add nonsense to wikipedia ...",0
96528,"""\n\n unblock request \n\n""",0
16019,"""\n\n unblock request \n\n""",0
29636,"""\n\n unblock request \n\n""",0
135029,"""\n\nplease do not add nonsense to wikipedia. ...",0
68816,"""\n\nplease do not add nonsense to wikipedia. ...",0
141796,"""thank you for experimenting with wikipedia. ...",0


Сгруппируем таблицу по дубликатам. Подсчитаем и сохраним в столбцах **text_count** и **toxic_sum** полное количество одинаковых сообщений (учитывая первое включение) и сумму значений целевого признака.

In [ ]:
dupls = comments[comments.text.duplicated(keep=False)].groupby('text').agg({'text' : ['count'], 'toxic' : ['sum']})
dupls.columns = ["_".join(x) for x in dupls.columns.to_numpy()]
dupls.head(5)

,text_count,toxic_sum
text,,
"""\n\n """"wikipedia isn't perfect, but its model is sound"""" \n\ncan i suggest that you two guys read professor john naughton's excellent article in """"the observer"""" of 10.08.14? if it helps you understand it, i can get it translated into belgian (french or walloon?) and into dutch? i use both wikipedia and odnb and both are equally 'intransigent': """"unwilling or refusing to change one's views or to agree about something.\nsynonyms: uncompromising, inflexible, unbending, unyielding, diehard, unshakable, unwavering, resolute, rigid, unaccommodating, uncooperative, stubborn, obstinate, obdurate, pigheaded, single-minded, iron-willed, stiff-necked""""; intransigeant; onverzettelijk: in french/dutch?\n\ni can debate the shortcomings of wikipedia until (as we say in colloquial english) """"the cows come home"""", but there is no complete agreement anywhere in the academic world as to who exactly constituted the bloomsbury group, so all the members are a matter of opinion and not fact guys.\n\nwhat is certain is that a significant proportion of male members were cambridge apostles.\n\nunderstand the above points and we may have the basis for further discussion? otherwise....2.27.131.74 """,2,0
"""\n\n please do not add nonsense to wikipedia \n\n as they say in the movies, """"oh be-have"""". seriously, though, we are trying to write an encyclopedia here, so don't make joke edits. some readers looking for a serious article might not find them amusing. remember, millions of people read wikipedia, so we have to take what we do a bit seriously here. if you'd like to experiment with editing, try the sandbox, where you can write whatever you want (as long as it's not offensive). """,2,0
"""\n\n unblock request \n\n""",3,0
"""\n\nplease do not add nonsense to wikipedia. it is considered vandalism. if you would like to experiment, use the sandbox. thank you. talk """,2,0
"""thank you for experimenting with wikipedia. your test worked, and it has been reverted or removed. please use the sandbox for any other tests you may want to do. take a look at the welcome page to learn more about contributing to our encyclopedia. talk """,2,0


Если для сообщения и его дубликатов целевой признак имеет значение 0, в столбец **toxic_sum** также будет 0, если 1, то в столеце **toxic_sum** будет значение равное полному количеству включений рассматриваемого сообщения в таблицу. Во всех других случаях, сообщение и хотя бы один из его дубликатов окажутся отнесенными к разным классам, что является ошибкой. Такие сообщения должны быть удалены из обучающей выборки.

In [ ]:
dupls['opposing_classes'] =\
    dupls.apply(lambda r: True if r.toxic_sum != 0 and r.toxic_sum != r.text_count else False, axis=1)
dupls[dupls.opposing_classes].head(5)

,text_count,toxic_sum,opposing_classes
text,,,
removing irrelevant nonsense\n\nad hominem attack in place of reasonable argument shows the bankruptcy of\nthose who which to silence me.\n\nmore you do more persistent i will be come.,2,1,True


Выведем одинаковые сообщения, отнесенные к разным классам.

In [ ]:
comments[comments.text.isin(dupls[dupls.opposing_classes].index)]

,text,toxic
80673,removing irrelevant nonsense\n\nad hominem att...,1
135632,removing irrelevant nonsense\n\nad hominem att...,0


Удалим этот текст из исходной таблицы, послего чего очистим ее от дубликатов сообщений.

In [ ]:
comments = comments[-comments.text.isin(dupls[dupls.opposing_classes].index)]
comments = comments.drop_duplicates(subset=['text'])
comments = comments.reset_index(drop=True)

Проверим наличие в корпусе киррилических символов/текстов.

In [ ]:
comments[comments.text.apply(lambda r: bool(re.search('[а-яё]', r)))].head(10)

,text,toxic
126,"ahh, hello witzeman \n\n203.92.84.161 \nsymbo...",0
228,http://www.users.bigpond.com/montdale/page8.ht...,0
449,"""\n\n yobot: incorrect defaultsort \n\nhi, yob...",0
996,it is common knowledge that karaims (but not k...,0
1907,"hi, 牛岩, and welcome to wikipedia! i hope you l...",0
2652,i understand your difficulty. i am able to rea...,0
3156,"""\n\nmaybe rather stop campaigning? my was the...",0
3797,"""this is the current smackdown song as of octo...",0
4235,"""\nno seriousy. the only thing i think actual...",1
4255,were those russian tourists by any chance drin...,0


Чего там только нет. Очистим текст сообщений от символов, не относящихся к английскому алфавиту.

In [ ]:
comments.text = comments.text.apply(lambda r: ' '.join(re.sub(r"[^a-z]", ' ', r).split()))

Лемматизируем текст.

In [ ]:
 comments['lemmatized'] = comments.text.apply(lemmatize)

Сравним строки до и после лемматизации.

In [ ]:
comments.head(10)

,text,toxic,lemmatized
0,explanation why the edits made under my userna...,0,explanation edits made username hardcore metal...
1,d aww he matches this background colour i m se...,0,aww match background colour seemingly stuck th...
2,hey man i m really not trying to edit war it s...,0,hey man really trying edit war guy constantly ...
3,more i can t make any real suggestions on impr...,0,make real suggestion improvement wondered sect...
4,you sir are my hero any chance you remember wh...,0,sir hero chance remember page
5,congratulations from me as well use the tools ...,0,congratulation well use tool well talk
6,cocksucker before you piss around on my work,1,cocksucker piss around work
7,your vandalism to the matt shirvington article...,0,vandalism matt shirvington article reverted pl...
8,sorry if the word nonsense was offensive to yo...,0,sorry word nonsense offensive anyway intending...
9,alignment on this subject and which are contra...,0,alignment subject contrary dulithgow


Интересно подсчитать количество строк, не изменившихся после лемматизации. Выведем их количество.

In [ ]:
(comments.text == comments.lemmatized).sum()

2953

Малая часть.

## Обучение

### Обучающая и тестовая выборки

Сформируем обучающую и тестовую выборки в соотношении 4:1.

In [ ]:
train, test = train_test_split(comments, shuffle=False, test_size=0.2, random_state=12345)

### Вычисление TD-IDF

Обучим TD-IDF векторизатор на обучающей выборке.

In [ ]:
lemmatized_corpus_train = train.lemmatized.values.astype('U')
lemmatized_corpus_test = test.lemmatized.values.astype('U')

vectorizer_train = TfidfVectorizer()
X_train = vectorizer_train.fit_transform(lemmatized_corpus_train)
y_train = train.toxic.values
#print(vectorizer_train.get_feature_names())
print(X_train.shape)

(127619, 138708)


Подготовим тестовые данные с помощью векторизатора, обученного на обучающей выборке.

In [ ]:
X_test = vectorizer_train.transform(lemmatized_corpus_test)
y_test = test.toxic.values
print(X_test.shape)

(31905, 138708)


### Обучение логистической регрессии на TD-IDF

Обучим логистическую регрессию на TD-IDF матрице, вычисленной для исследуемого копуса текста. Здесь и далее будем использовать стандартные параметры кросс-валидации GridSearchCV.

In [ ]:
# Скачиваем обученную модель
print(download_file_from_ydisk(MODEL_LR_PUBLIC_KEY, './model_lr'))

Файл https://disk.yandex.ru/d/77gZnQrqOsmZqA скачан и сохранен под именем ./model_lr


In [ ]:
if os.path.isfile('./model_lr'):
    lr = load('model_lr')
    print('Модель загружена из файла model_lr')
else:
    params = {
        'random_state': [12345],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'max_iter': [25, 100, 150],
        'class_weight': ['balanced', None]
    }
    lr = GridSearchCV(
        estimator=LogisticRegression(),
        param_grid=params,
        scoring='f1',
        refit=True,
        verbose=3
    )
    lr.fit(X_train, y_train)
    dump(lr, 'model_lr')
    print('Модель обучена и сохранена в файл model_lr')

Модель загружена из файла model_lr


Лучшее занчение f1-метрики на обучающей выборке и соответствующие гиперпараметры модели.

In [ ]:
lr.best_score_, lr.best_params_

(0.7605184708119831,
 {'class_weight': 'balanced',
  'max_iter': 25,
  'random_state': 12345,
  'solver': 'saga'})

Значение f1-метрики на тестовой выборке.

In [ ]:
f1_score(y_test, lr.predict(X_test))

0.7544532748698274

### Обучение LGBMClassifier на TD-IDF

Обучим классификатор LGBMClassifier.

In [ ]:
# Скачиваем обученную модель
print(download_file_from_ydisk(MODEL_LGBM_PUBLIC_KEY, './model_lgbm'))

Файл https://disk.yandex.ru/d/tQgentlE-9US-g скачан и сохранен под именем ./model_lgbm


In [ ]:
if os.path.isfile('./model_lgbm'):
    lgbm = load('model_lgbm')
    print('Модель загружена из файла model_lgbm')
else:
    params = {
        'random_state': [12345],
        'n_estimators': [150, 250, 350],
        'max_depth': [-1, 3],
        'class_weight': ['balanced', None]
    }
    lgbm = GridSearchCV(
        estimator=LGBMClassifier(),
        param_grid=params,
        scoring='f1',
        refit=True,
        verbose=10
    )
    lgbm.fit(X_train, y_train)
    dump(lgbm, 'model_lgbm')
    print('Модель обучена и сохранена в файл model_lgbm')

Модель загружена из файла model_lgbm


Лучшее занчение f1-метрики на обучающей выборке и соответствующие гиперпараметры модели.

In [ ]:
lgbm.best_score_, lgbm.best_params_

(0.7772148011998814,
 {'class_weight': None,
  'max_depth': -1,
  'n_estimators': 350,
  'random_state': 12345})

Значение f1-метрики на тестовой выборке.

In [ ]:
f1_score(y_test, lgbm.predict(X_test))

0.7803003842123647

### Обучение CatBoostClassifier на TD-IDF

Исследуем классификатор CatBoostClassifier.

In [ ]:
# Скачиваем обученную модель
print(download_file_from_ydisk(MODEL_CB_PUBLIC_KEY, './model_cb'))

Файл https://disk.yandex.ru/d/Ti8OKj54YNxFZg скачан и сохранен под именем ./model_cb


In [ ]:
if os.path.isfile('./model_cb'):
    cb = load('model_cb')
    print('Модель загружена из файла model_cb')
else:
    params = {
        'random_seed': [12345],
        'n_estimators': [150, 250, 350],
        #'max_depth': [None, 3]#,
        #'loss_function': ['Logloss', 'CrossEntropy']
    }
    cb = GridSearchCV(
        estimator=CatBoostClassifier(silent=True),
        param_grid=params,
        scoring='f1',
        refit=True,
        verbose=10
    )
    cb.fit(X_train, y_train)
    dump(cb, 'model_cb')
    print('Модель обучена и сохранена в файл model_cb')

Модель загружена из файла model_cb


Лучшее занчение f1-метрики на обучающей выборке и соответствующие гиперпараметры модели.

In [ ]:
cb.best_score_, cb.best_params_

(0.7484962011049803, {'n_estimators': 250, 'random_seed': 12345})

Значение f1-метрики на тестовой выборке.

In [ ]:
f1_score(y_test, cb.predict(X_test))

0.7495475931958017

## Выводы

Для оценки качества классификации тональности сообщений разными моделями машинного обучения был загружен и подготовлен корпус из 159571 сообщений. Данные не содержали пропусков, но имели незначительное количество дубликатов. На этапе предобработки дубликаты были удалены, при этом сообщения, хотя бы один дубликат которых имел противоположную классификацию, удалялись из корпуса вместе со всеми дубликатами.

Перед токенизацией сообщений и лемматизацией входивших в них слов все символы сообщений были переведены в нижний регистр, также все сообщения были очищены от символов, не являющимися буквами английского (латинского) алфавита.

Исходные данные были поделены на обучающую и тестовую в соотношении 4:1. Перед обучением моделей корпус текстов был преобразован TF-IDF векторизатором, обученным на обучающей выборке.

В качестве моделей были выбраны логистическая регрессия LogisticRegression модуля sklearn, а также классификаторы LGBMClassifier модуля lightgbm и CatBoostClassifier модуля catboost. Две последних библиотеки используют для обучения градиентый бустинг.

| Классификатор | F1-метрика на обучающих данных | F1-метрика на тестовых данных |
| :---------------------- | :---------------------- | :---------------------- |
| LGBMClassifier | 0.779 | 0.775 |
| CatBoostClassifier | 0.750 | 0.761 |
| LogisticRegression | 0.755 | 0.755 |

Полученные результаты приведены в таблице выше. По значению F1-меры на тестовых данных с незначительным отставанием вырвался вперед LGBMClassifier, использующий 350 предсказателей.

Отметим, что все библиотеки позволили преодолеть установленный минимум для F1- меры &mdash; 0.75.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны